# Loan Default Prediction – EDA & Data Cleaning Pipeline
This notebook performs data loading, exploratory analysis, missing value treatment, feature engineering, and dataset cleaning for the Loan Default Prediction project.


## 1. Import Required Libraries
This section imports the essential packages used for data manipulation and numerical operations.


In [1]:
import numpy as np
import pandas as pd


## 2. Load the Dataset
We load the raw loan acceptance dataset using `pd.read_csv()`. The file contains detailed borrower information used for loan default prediction.


In [32]:
data = pd.read_csv(r"C:\Users\gedal\Desktop\LoanDefaultPredictor\01_Data\01_Raw\accepted\accepted.csv",low_memory=False)

## 3. Initial Data Exploration

### 3.1 Preview Data  
Displays the first few rows to understand column structure and values.

### 3.2 Shape of the Dataset  
Shows number of rows and columns, which helps estimate dataset size.

### 3.3 Data Types & Memory Usage  
`data.info()` helps identify column types and potential columns needing cleaning.

### 3.4 Missing Value Overview  
Lists columns with the highest missing values, helping define the cleaning plan.


In [33]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
data.shape

(2260701, 151)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB


In [41]:
data.isnull().sum().sort_values(ascending=False).head(20)

member_id                                     2260701
orig_projected_additional_accrued_interest    2252050
hardship_reason                               2249784
hardship_payoff_balance_amount                2249784
hardship_last_payment_amount                  2249784
payment_plan_start_date                       2249784
hardship_type                                 2249784
hardship_status                               2249784
hardship_start_date                           2249784
deferral_term                                 2249784
hardship_amount                               2249784
hardship_dpd                                  2249784
hardship_loan_status                          2249784
hardship_length                               2249784
hardship_end_date                             2249784
settlement_status                             2226455
debt_settlement_flag_date                     2226455
settlement_term                               2226455
settlement_percentage       

## 4. Remove Hardship-Related Columns
These columns contain hardship program details, have extremely high missing values, and are not useful for default prediction. They are removed.


In [42]:
hardship_cols = [
    'orig_projected_additional_accrued_interest', 'hardship_reason',
    'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'payment_plan_start_date', 'hardship_type', 'hardship_status',
    'hardship_start_date', 'deferral_term', 'hardship_amount',
    'hardship_dpd', 'hardship_loan_status', 'hardship_length',
    'hardship_end_date'
]
data.drop(columns=hardship_cols, inplace=True)

## 5. Drop Columns With >95% Missing Values
Columns with more than 95% missing values are removed because they add no predictive value and increase noise.


In [44]:
threshold = 0.95  # 95%

cols_to_drop = data.columns[data.isnull().mean() > threshold]
data.drop(cols_to_drop, axis=1, inplace=True)

cols_to_drop

Index(['member_id', 'revol_bal_joint', 'sec_app_fico_range_low',
       'sec_app_fico_range_high', 'sec_app_earliest_cr_line',
       'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
       'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object')

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 117 entries, id to debt_settlement_flag
dtypes: float64(90), object(27)
memory usage: 2.0+ GB


In [47]:
for col in data.columns:
    if data[col].dtype == 'float64':
        data[col] = data[col].astype('float32')
    elif data[col].dtype == 'object':
        data[col] = data[col].astype('category')

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 117 entries, id to debt_settlement_flag
dtypes: category(27), float32(90)
memory usage: 1.0 GB


In [52]:
threshold = 0.99

low_variance_cols = []

for col in data.columns:
    top_freq = data[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_freq > threshold:
        low_variance_cols.append(col)

data.drop(columns=low_variance_cols, inplace=True)

print("Dropped low-variance columns:", low_variance_cols)
print("Remaining columns:", data.shape[1])


Dropped low-variance columns: []
Remaining columns: 111


In [53]:
data.isna().sum().sort_values(ascending=False).head(30)

verification_status_joint         2144971
dti_joint                         2139995
annual_inc_joint                  2139991
desc                              2134636
mths_since_last_record            1901545
mths_since_recent_bc_dlq          1741000
mths_since_last_major_derog       1679926
mths_since_recent_revol_delinq    1520342
next_pymnt_d                      1345343
mths_since_last_delinq            1158535
il_util                           1068883
mths_since_rcnt_il                 909957
all_util                           866381
open_acc_6m                        866163
total_cu_tl                        866163
inq_last_12m                       866163
open_rv_24m                        866162
open_rv_12m                        866162
open_il_24m                        866162
open_il_12m                        866162
total_bal_il                       866162
max_bal_bc                         866162
inq_fi                             866162
open_act_il                       

In [55]:
joint_cols = ['verification_status_joint', 'dti_joint', 'annual_inc_joint','desc']
data.drop(columns=joint_cols, inplace=True)

In [57]:
data['emp_title'] = data['emp_title'].cat.add_categories("Unknown").fillna("Unknown")

In [59]:
num_cols_with_na = data.select_dtypes(include=['float32','int32']).columns
for col in num_cols_with_na:
    data[col] = data[col].fillna(data[col].median())

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 104 entries, loan_amnt to debt_settlement_flag
dtypes: category(20), float32(84)
memory usage: 804.9 MB


In [68]:
print(data.columns.tolist())

['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'application_type', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12

In [66]:
data.drop(columns=['id','url','title'], inplace=True)

In [71]:
threshold=0.5*len(data)
null_cols = data.columns[data.isnull().sum() > threshold].tolist()
data.drop(columns=null_cols, inplace=True)

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 103 entries, loan_amnt to debt_settlement_flag
dtypes: category(19), float32(84)
memory usage: 802.7 MB


In [76]:
data.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,...,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06,2.260701e+06
mean,1.504690e+04,1.504163e+04,1.502341e+04,1.309282e+01,4.458059e+02,7.799223e+04,1.882344e+01,3.068708e-01,6.985882e+02,7.025884e+02,...,8.035826e-02,2.074368e+00,9.429807e+01,4.227047e+01,1.281142e-01,4.676824e-02,1.762541e+05,5.073154e+04,2.304112e+04,4.338879e+04
std,9.190183e+03,9.188350e+03,9.192269e+03,4.832103e+00,2.671717e+02,1.126953e+05,1.417788e+01,8.672199e-01,3.301015e+01,3.301102e+01,...,4.860507e-01,1.802067e+00,8.952810e+00,3.561802e+01,3.645142e-01,3.775225e-01,1.790733e+05,4.939347e+04,2.277300e+04,4.440789e+04
min,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,4.930000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,6.100000e+02,6.140000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+03,8.000000e+03,8.000000e+03,9.490000e+00,2.516500e+02,4.600000e+04,1.190000e+01,0.000000e+00,6.750000e+02,6.790000e+02,...,0.000000e+00,1.000000e+00,9.170000e+01,0.000000e+00,0.000000e+00,0.000000e+00,5.200400e+04,2.126100e+04,8.500000e+03,1.548800e+04
50%,1.290000e+04,1.287500e+04,1.280000e+04,1.262000e+01,3.779900e+02,6.500000e+04,1.784000e+01,0.000000e+00,6.900000e+02,6.940000e+02,...,0.000000e+00,2.000000e+00,1.000000e+02,3.750000e+01,0.000000e+00,0.000000e+00,1.142985e+05,3.786400e+04,1.630000e+04,3.269600e+04
75%,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,5.933200e+02,9.300000e+04,2.448000e+01,0.000000e+00,7.150000e+02,7.190000e+02,...,0.000000e+00,3.000000e+00,1.000000e+02,6.670000e+01,0.000000e+00,0.000000e+00,2.521970e+05,6.347100e+04,2.990000e+04,5.760100e+04
max,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,1.719830e+03,1.100000e+08,9.990000e+02,5.800000e+01,8.450000e+02,8.500000e+02,...,5.800000e+01,3.200000e+01,1.000000e+02,1.000000e+02,1.200000e+01,8.500000e+01,9.999999e+06,3.408095e+06,1.569000e+06,2.118996e+06


In [80]:
data.drop(columns='zip_code',inplace=True)


In [82]:
data.drop(columns='emp_title',inplace=True)

In [83]:
data.describe(include='category')

,term,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,disbursement_method,debt_settlement_flag
count,2260668,2260668,2260668,2113761,2260668,2260668,2260668,2260668,2260668,2260668,2260639,2260668,2258241,2260596,2260668,2260668,2260668
unique,2,7,35,11,6,3,139,9,14,51,754,2,136,141,2,2,2
top,36 months,B,C1,10+ years,MORTGAGE,Source Verified,Mar-2016,Fully Paid,debt_consolidation,CA,Sep-2004,w,Mar-2019,Mar-2019,Individual,Cash,N
freq,1609754,663557,145903,748005,1111450,886231,61992,1076751,1277877,314533,15400,1535467,853003,1371381,2139958,2182546,2226422


In [79]:
data['home_ownership'].value_counts()

home_ownership
MORTGAGE    1111450
RENT         894929
OWN          253057
ANY             996
OTHER           182
NONE             54
Name: count, dtype: int64

## 6. Feature Cleaning & Date Transformations
Several date columns are converted into datetime format.  
New features such as:
- `days_since_last_payment`
- `loan_age_months`
- `credit_history_length`
- `days_since_last_credit_pull`  
are created by subtracting dates from the current date.


In [85]:
# Convert relevant columns to datetime
data['issue_d'] = pd.to_datetime(data['issue_d'], format='%b-%Y', errors='coerce')
data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'], format='%b-%Y', errors='coerce')
data['last_pymnt_d'] = pd.to_datetime(data['last_pymnt_d'], format='%b-%Y', errors='coerce')
data['last_credit_pull_d'] = pd.to_datetime(data['last_credit_pull_d'], format='%b-%Y', errors='coerce')

# Reference date for calculations
reference_date = pd.to_datetime('2018-12-31')

# Formulas
data['loan_age_months'] = (data['issue_d'] - data['earliest_cr_line']).dt.days // 30
data['days_since_last_payment'] = (reference_date - data['last_pymnt_d']).dt.days
data['days_since_last_credit_pull'] = (reference_date - data['last_credit_pull_d']).dt.days
data['credit_history_length'] = (reference_date - data['earliest_cr_line']).dt.days // 30


In [86]:
data = data.drop(columns=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])


In [87]:
data.describe(include='category')

,term,grade,sub_grade,emp_length,home_ownership,verification_status,loan_status,purpose,addr_state,initial_list_status,application_type,disbursement_method,debt_settlement_flag
count,2260668,2260668,2260668,2113761,2260668,2260668,2260668,2260668,2260668,2260668,2260668,2260668,2260668
unique,2,7,35,11,6,3,9,14,51,2,2,2,2
top,36 months,B,C1,10+ years,MORTGAGE,Source Verified,Fully Paid,debt_consolidation,CA,w,Individual,Cash,N
freq,1609754,663557,145903,748005,1111450,886231,1076751,1277877,314533,1535467,2139958,2182546,2226422


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 101 entries, loan_amnt to credit_history_length
dtypes: category(13), float32(84), float64(4)
memory usage: 821.4 MB


In [89]:
print(data.columns.tolist())

['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'loan_status', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'application_type', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc

In [91]:
data = data.drop(columns=[
    'funded_amnt', 'funded_amnt_inv', 'sub_grade', 
    'out_prncp', 'out_prncp_inv', 
    'recoveries', 'collection_recovery_fee', 'mths_since_last_record', 'mths_since_last_major_derog'
])


In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 92 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   loan_amnt                       float32 
 1   term                            category
 2   int_rate                        float32 
 3   installment                     float32 
 4   grade                           category
 5   emp_length                      category
 6   home_ownership                  category
 7   annual_inc                      float32 
 8   verification_status             category
 9   loan_status                     category
 10  purpose                         category
 11  addr_state                      category
 12  dti                             float32 
 13  delinq_2yrs                     float32 
 14  fico_range_low                  float32 
 15  fico_range_high                 float32 
 16  inq_last_6mths                  float32 
 17  mths_sin

In [94]:
data.isnull().sum().sort_values(ascending=False).head(20)

emp_length                     146940
days_since_last_payment          2460
days_since_last_credit_pull       105
loan_age_months                    62
credit_history_length              62
loan_status                        33
addr_state                         33
verification_status                33
purpose                            33
term                               33
grade                              33
home_ownership                     33
initial_list_status                33
debt_settlement_flag               33
disbursement_method                33
application_type                   33
annual_inc                          0
mths_since_last_delinq              0
open_acc                            0
fico_range_high                     0
dtype: int64

## 7. Handling Missing Values

### Numerical Columns  
Median imputation is used to fill missing values:
- More robust to outliers  
- Preserves distribution better than mean  


In [96]:
num_cols = ['days_since_last_payment', 'days_since_last_credit_pull', 
            'loan_age_months', 'credit_history_length']
for col in num_cols:
    data[col].fillna(data[col].median())

### Categorical Columns  
Categorical features are filled with:
- `"Unknown"`  
- or most frequent category  
This ensures no missing values remain.


In [97]:
cat_cols = ['emp_length', 'loan_status', 'addr_state', 'verification_status',
            'purpose', 'term', 'grade', 'home_ownership', 
            'initial_list_status', 'debt_settlement_flag', 
            'disbursement_method', 'application_type']
for col in cat_cols:
    data[col].fillna(data[col].mode()[0], inplace=True)

C:\Users\gedal\AppData\Local\Temp\ipykernel_22672\877705480.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0], inplace=True)


## 8. Post-Cleaning Validation
We re-check:
- Missing values  
- Data types  
- Memory usage  
to confirm everything is clean and ready.

In [98]:
data.isnull().sum().sort_values(ascending=False).head(20)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
grade                     0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
addr_state                0
dti                       0
delinq_2yrs               0
fico_range_low            0
fico_range_high           0
inq_last_6mths            0
mths_since_last_delinq    0
open_acc                  0
pub_rec                   0
dtype: int64

In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 92 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   loan_amnt                       float32 
 1   term                            category
 2   int_rate                        float32 
 3   installment                     float32 
 4   grade                           category
 5   emp_length                      category
 6   home_ownership                  category
 7   annual_inc                      float32 
 8   verification_status             category
 9   loan_status                     category
 10  purpose                         category
 11  addr_state                      category
 12  dti                             float32 
 13  delinq_2yrs                     float32 
 14  fico_range_low                  float32 
 15  fico_range_high                 float32 
 16  inq_last_6mths                  float32 
 17  mths_sin

In [102]:
# Check shape and memory usage
print(data.shape)
print(data.info())

# Check for remaining nulls
print(data.isnull().sum().sort_values(ascending=False).head(20))


(2260701, 92)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 92 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   loan_amnt                       float32 
 1   term                            category
 2   int_rate                        float32 
 3   installment                     float32 
 4   grade                           category
 5   emp_length                      category
 6   home_ownership                  category
 7   annual_inc                      float32 
 8   verification_status             category
 9   loan_status                     category
 10  purpose                         category
 11  addr_state                      category
 12  dti                             float32 
 13  delinq_2yrs                     float32 
 14  fico_range_low                  float32 
 15  fico_range_high                 float32 
 16  inq_last_6mths                  float32 

## 9. Random Sampling Check
Displays 10 random rows to visually inspect cleaned data.


In [103]:
data.sample(10)

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method,debt_settlement_flag,loan_age_months,days_since_last_payment,days_since_last_credit_pull,credit_history_length
1312621,24000.0,60 months,15.610000,578.679993,C,4 years,MORTGAGE,63700.0,Source Verified,Charged Off,...,190546.0,109685.0,23300.0,86259.0,Cash,N,122.0,1309.0,698.0,181.0
1999991,12000.0,60 months,19.990000,317.859985,D,2 years,RENT,70000.0,Source Verified,Fully Paid,...,34738.0,27990.0,5300.0,27938.0,Cash,N,85.0,729.0,305.0,114.0
1684023,20000.0,60 months,14.990000,475.700012,C,6 years,MORTGAGE,68000.0,Source Verified,Fully Paid,...,194699.0,38606.0,4600.0,47355.0,Cash,N,222.0,121.0,-1.0,244.0
795024,20000.0,60 months,18.940001,518.159973,D,10+ years,RENT,45000.0,Verified,Current,...,28800.0,10204.0,22900.0,0.0,Cash,N,247.0,-60.0,-60.0,252.0
1677530,12000.0,36 months,11.390000,395.089996,B,10+ years,MORTGAGE,75000.0,Not Verified,Current,...,363750.0,74427.0,31400.0,50126.0,Cash,N,239.0,-60.0,-60.0,261.0
1063068,13300.0,60 months,21.969999,367.109985,E,6 years,OWN,35000.0,Verified,Charged Off,...,41008.0,28171.0,7600.0,25008.0,Cash,N,219.0,456.0,60.0,254.0
1625534,7000.0,36 months,8.490000,220.949997,A,3 years,RENT,65000.0,Source Verified,Fully Paid,...,114298.5,37864.0,16300.0,32696.0,Cash,N,410.0,2587.0,-32.0,503.0
744839,20000.0,36 months,5.320000,602.299988,A,10+ years,MORTGAGE,185000.0,Not Verified,Fully Paid,...,299007.0,57371.0,88000.0,30349.0,Cash,N,261.0,121.0,121.0,295.0
1877272,25000.0,36 months,7.900000,782.260010,A,3 years,RENT,82500.0,Verified,Fully Paid,...,52300.0,33339.0,39200.0,0.0,Cash,N,268.0,1126.0,1126.0,340.0
1431583,6000.0,36 months,11.060000,196.610001,B,10+ years,RENT,61000.0,Not Verified,Current,...,36465.0,15692.0,13000.0,17165.0,Cash,N,129.0,-60.0,-60.0,131.0


## 10. Save the Cleaned Dataset
The cleaned dataset is exported as a CSV file for modeling.


In [105]:
data.to_csv(r"C:\Users\gedal\Desktop\LoanDefaultPredictor\01_Data\02_Processed\LoanDefault_Cleaned.csv", index=False)
